## Task_18

##### Site:  https://erefdn.org

Utiliza o BeautifulSoup para processar a página.
Estrutura da página:
* Página principal: Possui uma lista com URLs para cada bolsa disponível
* Páginas dos Artigos: Uma página dedicada para cada bolsa, contendo todas as suas informações.




##### Processamento da página principal

O processamento da página principal percorre a lista de bolsas disponíveis, armazenando-as em uma lista.

<img src="imgs\principal.jpg" style="width: 500px;"/>

##### Processamento das páginas de cada bolsa (artigos)

Uma função encapsula todo o processamento de um artigo. Apenas a sua URL é passada.
O retorno ocorre através de um objeto JSON contendo os campos pertinentes.

<img src="imgs\artigo.jpg" style="width: 500px;"/>

### Imports

In [37]:
from bs4 import BeautifulSoup
import requests
import re
from lxml import html
from lxml import etree
import json
import pandas as pd
from time import sleep

### Definições Iniciais

In [41]:
url_pag_principal = 'https://erefdn.org/research-grants-projects/currently-funded-projects'

In [59]:
end_arq_csv_saida = './saida/resultado-raspagem-erefdn.csv'

In [43]:
tempo_espera_entre_chamadas =  0.25

### Funções Auxiliares

In [4]:
def processa_seletor(sel_bruto):
    """Processa a string bruta do seletor de CSS obtida no site para
    compatibilidade com a sitaxe esperada pelo Python/BeautifullSoup.
    Recebe:
            sel_bruto :: str
    Retorna:
            sel_processado :: str
    """
    
    sel_processado = sel_bruto.replace('nth-child', 'nth-of-type')
    return sel_processado

In [5]:
def obtem_soup(url):
    """Obtem o objeto Soup para uma URL
    Recebe:
            url :: str
    Retorna:
            soup :: bs4.BeautifulSoup
    """
    
    headers = {'User-Agent': 
               'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.text, 'lxml')
    
    return soup

In [6]:
def obtem_sel_raiz(soup):
    """Obtem a string de selecao raiz de CSS
    Utiliza a classe 'article' para localizar o id.
    Assume que ha apenas um artigo por url, o que eh o caso na pagina pesquisada.
    
    Recebe:
            soup :: bs4.BeautifulSoup
    Retorna:
            sel_raiz :: str
    
    Exemplo: <url> -> '#post-1353'
    """
    
    id_artigo = soup.find('article').get('id')
    sel_raiz = f'#{id_artigo}'
    
    return sel_raiz

In [7]:
def valor_int(str_valor):
    """Converte o valor financeiro da bolsa de string para float.
    Recebe:
            str_valor_usd :: str
    Retorna:
            valor_int :: int
    """
    chars_a_remover = ['$', 'U', 'S', ',', ' ']
    valor_inteiro = ''.join([char for char in str_valor if char not in chars_a_remover])
    return valor_inteiro

In [8]:
def remove_novas_linhas(str_bruta, sep='|'):
    """Remove novas linhas '\n's em excesso e 
    devolve uma string com separador=sep.
    Recebe:
            str_bruta :: str
    Retorna:
            str_proc  :: str
            
    Exemplo:
        str_bruta = '\n\nCampo 1:\n\n\nValor 1\n\nValor2\n\n\n'
        str_proc = '|Campo 1:|Valor 1|Valor2|'
    """
    return re.sub('\n+', sep, str_bruta).strip()

In [60]:
def gera_csv(lista_json, end_arq_csv_saida):
    """Cria um arquivo de saida no formato CSV.
    Recebe:
            lista_json :: list of jsons
    Retorna:
            arquivo de saida  :: arquivo csv
            df :: Pandas.DataFrame

    """
    print(f'Gerando o arquivo CSV de Saida em {end_arq_csv_saida}.')
    df = pd.DataFrame(lista_jsons, index=range(1, len(lista_jsons) + 1))
    
    df.to_csv(end_arq_csv_saida, sep=';', index=False)
    
    return df

### Funções de Parsing

In [9]:
def parse_cabecalho(soup, sel_raiz):
    """Parse do cabecalho, Titulo e Investigators
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            Titulo :: str
            Investigator :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(1) > div > div > div > div > div'

    sel_css_titulo = processa_seletor(f'{sel_raiz} > {sel_tronco} > h1')
    sel_css_instituicao = processa_seletor(f'{sel_raiz} > {sel_tronco} > div > p')
    
    # Obtem o titulo e o investigador
    titulo = soup.select(sel_css_titulo)[0].text
    
    try:
        investigador_bruto = soup.select(sel_css_instituicao)[0].text
    except:
        sel_css_instituicao = processa_seletor(f'{sel_raiz} > {sel_tronco} > p')
        investigador_bruto = soup.select(sel_css_instituicao)[0].text
        print('!!! ERRO !!! ')
        print(soup.select(sel_css_instituicao)[0].text)
    investigador = investigador_bruto.split(':')[1].strip()
    
    return {'titulo': titulo, 'instituicao': investigador}

In [10]:
def parse_data_valor(soup, sel_raiz):
    """Parse do Data de Inicio, Valor da bolsa
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            Data :: str
            Valor :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(2)'
    sel_css_data = processa_seletor(f'{sel_raiz} > {sel_tronco} > div:nth-child(1)')
    sel_css_valor = processa_seletor(f'{sel_raiz} > {sel_tronco} > div:nth-child(2)')
    
    # Obtem a data inicial e o valor
    data_ini_bruta = soup.select(sel_css_data)[0].text
    try:
        data_ini = remove_novas_linhas(data_ini_bruta, sep='|').split('|')[2]
    except:
        data_ini = 'N/A'
    
    valor_bruto = soup.select(sel_css_valor)[0].text
    valor_str  = remove_novas_linhas(valor_bruto, sep='|').split('|')[2]
    valor_num = valor_int(valor_str)
    
    return {'data_ini': data_ini, 'valor_bolsa': valor_num}

In [11]:
def parse_descricao_detalhada(soup, sel_raiz):
    """Parse da descricao detalhada 
    Recebe: 
            soup do artigo
            seletor css para o artigo
    Retorna:
            descricao_detalhada :: str
    """
    # Processa o seletor de CSS:
    sel_tronco =  'section > div:nth-child(3)'
    sel_css_descr_det = processa_seletor(f'{sel_raiz} > {sel_tronco}')

    # Obtem a descricao detalhada 
    descricao_detalhada_bruta = soup.select(sel_css_descr_det)[0].text
    descricao_detalhada = remove_novas_linhas(descricao_detalhada_bruta, sep='\n')
  
    return {'descr_detalhada': descricao_detalhada}

In [12]:
def parse_lista_bolsas(soup, sel_raiz):
    """Parse da pagina principal com 
       a lista de bolsas a ser coletada.
    Recebe: 
            soup da pagina 
            seletor css para o artigo
    Retorna:
            json contendo as seguintes informacoes para cada bolsa disponivel:
                titulo :: str
                url_artigo :: str
    """

    # Obtem a lista de bolsas disponiveis
    lista_bolsas = []
    tags_bolsas = soup.select(sel_raiz)[0]
    
    # Para cada bolsa, isola o link do artigo e adiciona à lista de retorno
    for el in tags_bolsas.findAll('a'):
        titulo = el.text
        url_artigo = el.get('href')

        lista_bolsas.append({'titulo': titulo, 'url_artigo': url_artigo})
  
    return lista_bolsas

In [75]:
def parse_artigo(soup, url_artigo, sel_raiz_artigo):
    """A partir do Soup e do seletor raiz do artigo, faz seu parsing
    devolvendo um objeto JSON com os campos pertinentes.
    Recebe: 
            soup do artigo
            url do artigo
            seletor css para o artigo
    Retorna:
            json contendo as seguintes informacoes:
                titulo :: str
                instituicao :: str
                data_ini :: str
                valor_bolsa :: int (USD)
                descricao_detalhada :: str
                links :: str
    """
    campos = {}
    
    # Obtem titulo e instituicao
    campos.update(parse_cabecalho(soup, sel_raiz_artigo))
    
    # Obtem data_ini e valor
    campos.update(parse_data_valor(soup, sel_raiz_artigo))
    
    # Obtem descr_detalhada
    campos.update(parse_descricao_detalhada(soup, sel_raiz_artigo))
    
    # Inclui a url do artigo
    campos.update({'links': url_artigo})
    
    json_campos = json.loads(json.dumps(campos))
    return json_campos

### Funções Processamento Completo

In [58]:
def processa_artigo(url_artigo):
    """Processamento completo de um artigo 
    Recebe: 
            url do artigo :: str
    Retorna:
            json contendo as seguintes informacoes:
                titulo :: str
                instituicao :: str
                data_ini :: str
                valor_bolsa :: int (USD)
                descricao_detalhada :: str
                links :: str
    """
    # Obtem o objeto soup para uma bolsa especifica
    soup_artigo = obtem_soup(url_artigo)
    sel_css_artigo = obtem_sel_raiz(soup_artigo)
    
    # Obtem o json com o resultado do parsing
    json_artigo = parse_artigo(soup_artigo, url_artigo, sel_css_artigo)
    
    return json_artigo

In [81]:
def processa_pag_principal(url_pag_principal, log=False):
    """Processamento completo da pagina. 
    Gera a lista de bolsas disponiveis. Para cada bolsa, 
        faz o processamento completo dos dados.
    Recebe: 
            url do artigo :: str
    Retorna:
            lista de JSONs com o parsing de cada bolsa.
    """
    # Obtem o objeto soup para a pagina inicial, 
    # com links para todas as bolsas oferecidas
    soup_pag_principal = obtem_soup(url_pag_principal)
    sel_css_pag_princ = obtem_sel_raiz(soup_pag_principal)
    
    # Obtem a lista de bolsas
    lista_bolsas = parse_lista_bolsas(soup_pag_principal, sel_css_pag_princ)
    lista_jsons = []
    
    # Para cada bolsa, processa o artigo correspondente, 
    # que fica em uma pag separada.
    for i, bolsa in enumerate(lista_bolsas):
        # Obtem informacoes basicas de cada bolsa
        titulo_artigo = bolsa['titulo']
        url_artigo = bolsa['url_artigo']
        print(f'---- Processando artigo {i + 1:2d}.. {titulo_artigo[:min(60, len(titulo_artigo))]}..')
        
        # Obtem o parsing completo da bolsa
        json_artigo = processa_artigo(url_artigo)
        
        # Salva o resultado na lista de jsons
        lista_jsons.append(json_artigo)
        
        # Imprime resultado na tela se a flag de log for True
        if log == True:
            print(json.dumps(json_artigo, indent=4))
        
        sleep(tempo_espera_entre_chamadas)

    return lista_jsons


### Execução

In [80]:
# Obtem a lista de parsings para cada bolsa
lista_jsons = processa_pag_principal(url_pag_principal)

---- Processando artigo  1.. Non-Recyclable Plastics to Pavements..
---- Processando artigo  2.. Techno-Economic Evaluation of Supercritical Water Oxidation for Landfi..
---- Processando artigo  3.. Effectiveness of Landfill Liners to Control Transport of PFAS..
---- Processando artigo  4.. Repair Strategies for Waste Transfer Station Concrete Overlay..
---- Processando artigo  5.. Innovative Technologies to Treat Per- and Polyfluoroalkyl Substances (..
---- Processando artigo  6.. Polymer-Based Pre-Treatment for Removal of PFAS from Landfill Leachate..
---- Processando artigo  7.. The Influence of Social Norms on Recycling Behavior in Urban Multifami..
---- Processando artigo  8.. Recycling, Contamination, Markets and MRFs: Practical Strategies for C..
---- Processando artigo  9.. Development of Recognizable Recycled Paper Based Containerboard Produc..
---- Processando artigo 10.. Recovering High Value Acids from Anaerobic Co-digestion of Municipal S..
!!! ERRO !!! 
Investigators: Nor

In [82]:
# Gera o arquivo CSV de saida
df = gera_csv(lista_jsons, end_arq_csv_saida)

Gerando o arquivo CSV de Saida em ./saida/resultado-raspagem-erefdn.csv.


In [83]:
df

,titulo,instituicao,data_ini,valor_bolsa,descr_detalhada,links
1,Non-Recyclable Plastics to Pavements,University of Illinois Urbana-Champaign,TBD,161075,This proposal seeks to create high-value and h...,https://erefdn.org/non-recyclable-plastics-to-...
2,Techno-Economic Evaluation of Supercritical Wa...,Duke University,Oct 2020,152000,Landfill leachate and condensate management ca...,https://erefdn.org/techno-economic-evaluation-...
3,Effectiveness of Landfill Liners to Control Tr...,University of Virginia,Jan 2020,142000,The efficacy of Subtitle D landfill lining sys...,https://erefdn.org/effectiveness-of-landfill-l...
4,Repair Strategies for Waste Transfer Station C...,North Carolina State University,Mar 2020,195000,"Recently, it has been shown that the premature...",https://erefdn.org/repair-strategies-for-waste...
5,Innovative Technologies to Treat Per- and Poly...,Clarkson University,February 2020,145000,Disposing of leachate-containing per- and poly...,https://erefdn.org/innovative-technologies-to-...
6,Polymer-Based Pre-Treatment for Removal of PFA...,Geosyntec,March 2020,105000,Per- and polyfluoroalkyl substances (PFAS) are...,https://erefdn.org/polymer-based-pre-treatment...
7,The Influence of Social Norms on Recycling Beh...,Stony Brook University,,,The primary goal of this proposal is to test t...,https://erefdn.org/the-influence-of-social-nor...
8,"Recycling, Contamination, Markets and Material...","Skumatz Economic Research Associates, Inc.",,,Material Recovery Facilities (MRFs) can techni...,https://erefdn.org/recycling-contamination-mar...
9,Development of Recognizable Recycled Paper Bas...,North Carolina State University,N/A,,There is a growing interest in sustainable pac...,https://erefdn.org/development-of-recognizable...
10,Recovering High Value Acids from Anaerobic Co-...,North Carolina State University and University...,May 2019,16700,Re-engineering the anaerobic co-digestion (co-...,https://erefdn.org/recovering-high-value-acids...
